In [1]:
import pandas as pd
import json
import os
import ast
import re
import numpy as np
import matplotlib.pyplot as plt
import torch
from time import time
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, f1_score,  recall_score, confusion_matrix, precision_score
from transformers import (
    set_seed,
)
set_seed(42)

from unsloth import FastLanguageModel
max_seq_length = 500
from tqdm import tqdm

import pickle

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
pd.set_option("display.max_rows", None)  # Show all rows
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", None)  # Adjust width to fit content
pd.set_option("display.colheader_justify", "left")  # Align headers
pd.set_option("display.max_colwidth", None)  # Prevent column truncation


In [3]:
if os.getcwd() == '/root':
    new_path = "/root/0_Thesis/0_final/"
    os.chdir(new_path)
else:
    os.chdir("..") 
print(os.getcwd())

/root/0_Thesis/0_final


# Section 1: Binary

In [4]:
results_binary = pickle.load(open("6_evaluation/all_binary_result.pkl", "rb"))
len(results_binary)

71

In [5]:
reprot = pd.DataFrame(columns=["auc", "acc", "P", "R", "F1"])
for k,v in results_binary.items():
    score_all = []
    all_probs = []
    all_y_true = []
    
    for random_state, data in v.items():
        all_probs.extend(data["probs_value_1"])
        all_y_true.extend(data["y_true"])

    all_probs = np.array(all_probs)
    all_y_true = np.array(all_y_true)
    score_all.append(round(roc_auc_score(np.array(all_y_true), np.array(all_probs)), 3))

    all_y_pred = all_probs >= 0.3
    score_all.append(round(accuracy_score(all_y_true , all_y_pred), 3))
    score_all.append(round(precision_score(all_y_true , all_y_pred), 3))
    score_all.append(round(recall_score(all_y_true , all_y_pred), 3))
    score_all.append(round(f1_score(all_y_true, all_y_pred), 3))
    reprot.loc[k] = score_all


/root/envs/ros_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [6]:
print(reprot)

                      auc    acc    P      R      F1   
m2.v2.vote2.14B       0.789  0.656  0.821  0.300  0.440
m2.v5.14B.14B         0.798  0.716  0.676  0.709  0.692
m2.v2.lgb.14B         0.800  0.707  0.757  0.514  0.612
Qwen0.5               0.473  0.481  0.432  0.489  0.459
m2.v1.Qwen0.5B        0.688  0.585  0.716  0.128  0.217
m2.v1.Gemma2B         0.607  0.597  0.561  0.479  0.517
gemma2b               0.619  0.597  0.626  0.484  0.546
Gemma9B               0.673  0.628  0.585  0.591  0.588
m1.v3.lgb.llama1B     0.664  0.614  0.651  0.308  0.418
m1.v4.lgb.llama1B     0.694  0.604  0.763  0.174  0.283
m2.v1.14B.llama1B     0.724  0.663  0.625  0.623  0.624
m2.v3.lgb.Llama1B     0.701  0.639  0.580  0.716  0.641
m1.v5.lgb.Llama1B     0.675  0.593  0.803  0.124  0.215
m2.v2.mean.Llama1B    0.732  0.673  0.655  0.578  0.614
m1.v1.vote3.Llama1B   0.678  0.621  0.594  0.497  0.541
m2.v1.9B.Llama1B      0.672  0.637  0.625  0.485  0.546
m2.v1.7B.Llama1B      0.707  0.550  0.000  0.000